In [2]:
import os
import numpy as np
import pandas as pd
from skimage import io, color, feature
from skimage.feature import graycomatrix, graycoprops

# Function to calculate GLCM features for a single image
def calculate_glcm_features(image_path):
    # Read image
    image = io.imread(image_path)

    # Convert image to grayscale
    if len(image.shape) > 2:
        gray_image = color.rgb2gray(image)
    else:
        gray_image = image

    # Convert to unsigned integer type
    gray_image = (gray_image * 255).astype(np.uint8)

    # Calculate GLCM
    distances = [1, 2, 3]  # Define distances
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Define angles
    glcm = graycomatrix(gray_image, distances=distances, angles=angles, levels=256,
                        symmetric=True, normed=True)

    # Calculate GLCM properties
    properties = ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']
    glcm_features = {prop: graycoprops(glcm, prop).ravel() for prop in properties}

    return glcm_features


# Function to convert TIF images to CSV
def tif_to_csv(input_folder, output_csv):
    # Initialize list to store GLCM features dataframes
    dfs = []

    # Iterate through each TIF image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif'):
            image_path = os.path.join(input_folder, filename)

            # Calculate GLCM features for the current image
            glcm_features = calculate_glcm_features(image_path)

            # Convert GLCM features to dataframe and append to list
            dfs.append(pd.DataFrame(glcm_features))

    # Concatenate all dataframes in the list
    df = pd.concat(dfs, ignore_index=True)

    # Save dataframe to CSV
    df.to_csv(output_csv, index=False)

# Example usage
input_folder = '/content/drive/MyDrive/nagapatinam/Hyper'
output_csv = '/content/drive/MyDrive/nagapatinam/hyperspectral.csv'
tif_to_csv(input_folder, output_csv)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/nagapatinam/hyperspectral.csv")

In [4]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,2].astype('int')
print(x_feature.shape)
print(y_label.shape)


(7908, 4)
(7908,)


In [5]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tqdm import tqdm


In [6]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.10,random_state=7)


In [7]:
# Normalise the data after splitting to avoid information leak between train and test set.

scaler_norm = MinMaxScaler()

X_train = scaler_norm.fit_transform(X_train)
X_test = scaler_norm.fit_transform(X_test)


In [8]:
from sklearn.ensemble import RandomForestClassifier
train_x,test_x,train_y,test_y = train_test_split(x_feature,y_label,test_size = 0.30)
rfc = RandomForestClassifier(n_estimators=4000,criterion='gini')
rfc = rfc.fit(train_x,train_y)
y_pred3 = rfc.predict(test_x)
print("Accuracy Score of Random Forest Classifier : ", accuracy_score(y_pred3,test_y))


Accuracy Score of Random Forest Classifier :  1.0


In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
train_x, test_x, train_y, test_y = train_test_split(x_feature, y_label, test_size=0.30)
dtc = DecisionTreeClassifier(criterion='gini')
dtc.fit(train_x, train_y)
y_pred = dtc.predict(test_x)
accuracy = accuracy_score(test_y, y_pred)
print("Accuracy Score of Decision Tree Classifier:", accuracy)


Accuracy Score of Decision Tree Classifier: 1.0
